In [52]:
import subprocess
from shlex import split
import os
from hashlib import md5
from functools import partial


bdir = '/home/n.zabarin/mpi/cp'
source = os.path.join(bdir, 'main.cpp')
executable = os.path.join(bdir, 'cp')
generator = os.path.join(bdir, 'gen.py')
outfile = os.path.join(bdir, 'output')
routfile = os.path.join(bdir, 'routput')

def build():
    cmd = "mpic++ --std=c++11 {} -o {}".format(source, executable)
    print(cmd)
    subprocess.run(split(cmd))
    
def gen(**kwargs):
    kwargs['file'] = os.path.join(bdir, kwargs['file'])
    cmd = """python3 {gen} {file} {x} "{offset}" "{ln}" "{value}" """.format(gen=generator, **kwargs)
    print(cmd)
    subprocess.run(split(cmd))
    
def run(np=4, x=256, y=256, iter_n=10000, err=0.0001, file='test'):
    file = os.path.join(bdir, file)
    
    cmd = "mpirun -q -n {} {} ".format(np, executable)    
    cmd += '{} {} {} {} "{}"'.format(x, y, iter_n, err, file)
    
    print(cmd)
    subprocess.run(split(cmd))
    
def _md5_file(filename):
    with open(filename, mode='rb') as f:
        d = md5()
        for buf in iter(partial(f.read, 128), b''):
            d.update(buf)
    return d.hexdigest()

def check(file=outfile):
    return _md5_file(file)

def full_test(name, iter_n, err, x, offset, ln, value, y=None):
    if not y: 
        y = x
        
    tests = [1, 2, 3, 4, 8, 16]
    
    build()
    
    ldir = os.path.join(bdir, name)
    os.mkdir(ldir)
    
    test_file = 'input'
    test_file = os.path.join(name, test_file)
    
    gen(file=test_file, x=x, offset=offset, ln=ln, value=value)
    
    chks = []
    
    for np in tests:
        run(np=np, x=x, y=y, iter_n=iter_n, err=err, file=test_file)
        chks.append(check())
        os.rename(outfile, os.path.join(ldir, 'output_{}'.format(np)))
        os.rename(routfile, os.path.join(ldir, 'routput_{}'.format(np)))
        
    return chks
        



In [69]:
import plotly.plotly as py
import plotly.graph_objs as go

def c(a):
    a = float(a)
    if a < 0.000001:
        return -1
    return a

def draw(file, x, name='name'):
    with open(file, 'r') as f:
        z = [list(map(c, f.readline().split())) for i in range(x)]
        
    data = [go.Heatmap(z=z)]
    return py.iplot(data, filename=name)
    

mpirun -q -n 4 /home/n.zabarin/mpi/cp/cp 256 256 10000 0.0001 "/home/n.zabarin/mpi/cp/test"


In [70]:
full_test('try2', 10000, 0.0001, 256, "abs(x-128)", "256-abs(x-128)", "x/256")
draw('/home/n.zabarin/mpi/cp/try2/routput_1', 256, 'try2_1')

mpic++ --std=c++11 /home/n.zabarin/mpi/cp/main.cpp -o /home/n.zabarin/mpi/cp/cp
python3 /home/n.zabarin/mpi/cp/gen.py /home/n.zabarin/mpi/cp/try2/input 256 "abs(x-128)" "256-abs(x-128)" "x/256" 
mpirun -q -n 1 /home/n.zabarin/mpi/cp/cp 256 256 10000 0.0001 "/home/n.zabarin/mpi/cp/try2/input"
mpirun -q -n 2 /home/n.zabarin/mpi/cp/cp 256 256 10000 0.0001 "/home/n.zabarin/mpi/cp/try2/input"
mpirun -q -n 3 /home/n.zabarin/mpi/cp/cp 256 256 10000 0.0001 "/home/n.zabarin/mpi/cp/try2/input"
mpirun -q -n 4 /home/n.zabarin/mpi/cp/cp 256 256 10000 0.0001 "/home/n.zabarin/mpi/cp/try2/input"
mpirun -q -n 8 /home/n.zabarin/mpi/cp/cp 256 256 10000 0.0001 "/home/n.zabarin/mpi/cp/try2/input"
mpirun -q -n 16 /home/n.zabarin/mpi/cp/cp 256 256 10000 0.0001 "/home/n.zabarin/mpi/cp/try2/input"


/opt/ansocial/env/lib/python3.5/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [79]:
full_test('try3', 10000, 0.0001, 256, "(x-128)**2//128", "max(1, 256-((x-128)**2//64))", "x/256")
draw('/home/n.zabarin/mpi/cp/try3/routput_1', 256, 'try3_1')

mpic++ --std=c++11 /home/n.zabarin/mpi/cp/main.cpp -o /home/n.zabarin/mpi/cp/cp
python3 /home/n.zabarin/mpi/cp/gen.py /home/n.zabarin/mpi/cp/try3/input 256 "(x-128)**2//128" "max(1, 256-((x-128)**2//64))" "x/256" 
mpirun -q -n 1 /home/n.zabarin/mpi/cp/cp 256 256 10000 0.0001 "/home/n.zabarin/mpi/cp/try3/input"
mpirun -q -n 2 /home/n.zabarin/mpi/cp/cp 256 256 10000 0.0001 "/home/n.zabarin/mpi/cp/try3/input"
mpirun -q -n 3 /home/n.zabarin/mpi/cp/cp 256 256 10000 0.0001 "/home/n.zabarin/mpi/cp/try3/input"
mpirun -q -n 4 /home/n.zabarin/mpi/cp/cp 256 256 10000 0.0001 "/home/n.zabarin/mpi/cp/try3/input"
mpirun -q -n 8 /home/n.zabarin/mpi/cp/cp 256 256 10000 0.0001 "/home/n.zabarin/mpi/cp/try3/input"
mpirun -q -n 16 /home/n.zabarin/mpi/cp/cp 256 256 10000 0.0001 "/home/n.zabarin/mpi/cp/try3/input"


/opt/ansocial/env/lib/python3.5/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [80]:
full_test('try4', 10000, 0.0001, 256, "0", "256", "x/256")
draw('/home/n.zabarin/mpi/cp/try4/routput_1', 256, 'try4_1')

mpic++ --std=c++11 /home/n.zabarin/mpi/cp/main.cpp -o /home/n.zabarin/mpi/cp/cp
python3 /home/n.zabarin/mpi/cp/gen.py /home/n.zabarin/mpi/cp/try4/input 256 "0" "256" "x/256" 
mpirun -q -n 1 /home/n.zabarin/mpi/cp/cp 256 256 10000 0.0001 "/home/n.zabarin/mpi/cp/try4/input"
mpirun -q -n 2 /home/n.zabarin/mpi/cp/cp 256 256 10000 0.0001 "/home/n.zabarin/mpi/cp/try4/input"
mpirun -q -n 3 /home/n.zabarin/mpi/cp/cp 256 256 10000 0.0001 "/home/n.zabarin/mpi/cp/try4/input"
mpirun -q -n 4 /home/n.zabarin/mpi/cp/cp 256 256 10000 0.0001 "/home/n.zabarin/mpi/cp/try4/input"
mpirun -q -n 8 /home/n.zabarin/mpi/cp/cp 256 256 10000 0.0001 "/home/n.zabarin/mpi/cp/try4/input"
mpirun -q -n 16 /home/n.zabarin/mpi/cp/cp 256 256 10000 0.0001 "/home/n.zabarin/mpi/cp/try4/input"


/opt/ansocial/env/lib/python3.5/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead

